In [1]:
%matplotlib inline

In [2]:
# importing necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import zscore
import statistics as stats
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor


In [3]:
# read sales data as 'df'
df = pd.read_csv('wallmart.csv')

,Year,Month,ProductCategory,Sales(In ThousandDollars)
0,2009,1,WomenClothing,1755.0
1,2009,1,MenClothing,524.0
2,2009,1,OtherClothing,936.0
3,2009,2,WomenClothing,1729.0
4,2009,2,MenClothing,496.0


In [54]:
df.head()

,Year,Month,ProductCategory,Sales
0,2009,1,1,1755.0
1,2009,1,2,524.0
2,2009,1,3,936.0
3,2009,2,1,1729.0
4,2009,2,2,496.0


In [4]:
#read excel file with multiple sheets
file = pd.ExcelFile('weather.xlsx')
file.sheet_names

['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

In [5]:
# checking shape of each sheet
for i in file.sheet_names:
    data = pd.read_excel(file, sheet_name= i )
    print(data.shape)


(365, 23)
(365, 23)
(365, 23)
(366, 23)
(365, 23)
(365, 23)
(365, 23)
(366, 23)


In [6]:
# read weather data excel sheet into df_weather
df_weather = pd.read_excel('weather.xlsx', sheetname=None, ignore_index=True)

C:\Users\noah\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [7]:
# concat all sheets into one
cdf_weather = pd.concat(df_weather.values())

cdf_weather.head()

,Year,Month,Day,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,...,Sea Level Press. (hPa) avg,Sea Level Press. (hPa) low,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent
0,2009,Jan,1,-3,-6,-9,-16,-17,-19,54,...,1023,1015,16,16,16,37,18,60,0,NaN
1,2009,Jan,2,1,-2,-5,-3,-7,-17,78,...,1012,1007,16,13,2,27,10,48,T,Snow
2,2009,Jan,3,3,1,-2,-5,-9,-13,72,...,1015,1008,16,16,16,27,16,42,T,NaN
3,2009,Jan,4,6,1,-4,-10,-12,-13,55,...,1017,1015,16,16,16,32,12,40,0,NaN
4,2009,Jan,5,6,5,3,-1,-5,-16,62,...,1014,1013,16,16,16,23,11,34,T,NaN


# Preprocessing sales data

In [8]:
df.rename(columns = {'Sales(In ThousandDollars)' : 'Sales'}, inplace= True)

In [9]:
df['ProductCategory'] = df['ProductCategory'].astype('category', categories = df['ProductCategory'].unique( ))

C:\Users\noah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.


In [10]:
df['ProductCategory'].replace(['OtherClothing','MenClothing','WomenClothing'] , [3,2,1], inplace = True)

In [11]:
df.isna().apply(pd.value_counts)

,Year,Month,ProductCategory,Sales
False,216.0,216.0,216.0,204
True,NaN,NaN,NaN,12


In [12]:
# simple loop to replace NaN values with mean of adjacent sales of same category
x = []
for i in range(0,len(df.Sales)):
    if df.isna().Sales[i] == True :
        x.append(stats.mean([df.Sales[i+3],df.Sales[i-3]]))
        
    else:
        x.append(df.Sales[i])
Sales = pd.Series(x)


In [13]:
df['Sales'] = Sales

In [14]:
df.isna().apply(pd.value_counts)

,Year,Month,ProductCategory,Sales
False,216,216,216,216


In [15]:
# Final sales data
df.head()

,Year,Month,ProductCategory,Sales
0,2009,1,1,1755.0
1,2009,1,2,524.0
2,2009,1,3,936.0
3,2009,2,1,1729.0
4,2009,2,2,496.0


In [16]:
# Final sales data
df.head()

(216, 4)

# Preprocessing weather data

In [17]:
cdf_weather.head()

,Year,Month,Day,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,...,Sea Level Press. (hPa) avg,Sea Level Press. (hPa) low,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent
0,2009,Jan,1,-3,-6,-9,-16,-17,-19,54,...,1023,1015,16,16,16,37,18,60,0,NaN
1,2009,Jan,2,1,-2,-5,-3,-7,-17,78,...,1012,1007,16,13,2,27,10,48,T,Snow
2,2009,Jan,3,3,1,-2,-5,-9,-13,72,...,1015,1008,16,16,16,27,16,42,T,NaN
3,2009,Jan,4,6,1,-4,-10,-12,-13,55,...,1017,1015,16,16,16,32,12,40,0,NaN
4,2009,Jan,5,6,5,3,-1,-5,-16,62,...,1014,1013,16,16,16,23,11,34,T,NaN


In [18]:
cdf_weather.fillna(value = 'Normal', inplace = True)

In [19]:
cdf_weather.replace(to_replace= '-', value= np.NaN , inplace= True)


In [20]:
cdf_weather.replace(to_replace= 'T', value= np.NaN , inplace= True)

In [21]:
cdf_weather.isna().apply(pd.value_counts)

,Year,Month,Day,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,...,Sea Level Press. (hPa) avg,Sea Level Press. (hPa) low,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent
False,2922.0,2922.0,2922.0,2920,2920,2920,2920,2920,2920,2920,...,2903,2903,2894,2894,2894,2902,2902,2863,2716,2922.0
True,NaN,NaN,NaN,2,2,2,2,2,2,2,...,19,19,28,28,28,20,20,59,206,NaN


In [22]:
cdf_weather = cdf_weather.interpolate()

In [23]:
cdf_weather.isna().apply(pd.value_counts)

,Year,Month,Day,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,...,Sea Level Press. (hPa) avg,Sea Level Press. (hPa) low,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent
False,2922,2922,2922,2922,2922,2922,2922,2922,2922,2922,...,2922,2922,2922,2922,2922,2922,2922,2922,2922,2922


In [24]:
cdf_weather.Month.replace( to_replace= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'] , value = [1,2,3,4,5,6,7,8,9,10,11,12], inplace = True)

# Only with month-wise sales data

In [92]:
df.head()

,Year,Month,ProductCategory,Sales
0,2009,1,1,1755.0
1,2009,1,2,524.0
2,2009,1,3,936.0
3,2009,2,1,1729.0
4,2009,2,2,496.0


In [93]:
X = df.iloc[:,0:3]
y = df.Sales

In [94]:
DT = DecisionTreeRegressor(max_depth= 7)
DT.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [95]:
predicted_labels = DT.predict(X)
rms = sqrt(mean_squared_error(y, predicted_labels))
rms

70.3705508778317

In [29]:
predicted_labels.shape

(216,)

# Validation

In [87]:
from sklearn.model_selection import train_test_split

X_trainv, X_testv, y_trainv, y_testv = train_test_split(df, y, test_size = 0.25, random_state = 21)

In [113]:
DTval = DecisionTreeRegressor(max_depth= 8, random_state= 5)
DTval.fit(X_trainv,y_trainv)

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=5, splitter='best')

In [114]:
pred_val = DTval.predict(X_testv)
rms_val = sqrt(mean_squared_error(y_testv, pred_val))
rms_val

51.39223087231106

# Test

In [37]:
test = pd.read_csv('test.csv')

In [44]:
test.head()

,Year,Month,ProductCategory,Sales
0,2015,1,1,NaN
1,2015,1,2,NaN
2,2015,1,3,NaN
3,2015,2,1,NaN
4,2015,2,2,NaN


In [39]:
test.rename(columns = {'Sales(In ThousandDollars)' : 'Sales'}, inplace= True)

In [40]:
test['ProductCategory'] = test['ProductCategory'].astype('category', categories = test['ProductCategory'].unique( ))

C:\Users\noah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """Entry point for launching an IPython kernel.


In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
Year               36 non-null int64
Month              36 non-null int64
ProductCategory    36 non-null category
Sales              0 non-null float64
dtypes: category(1), float64(1), int64(2)
memory usage: 1.1 KB


In [43]:
test['ProductCategory'].replace(to_replace =  ['OtherClothing','MenClothing','WomenClothing'] , value = [3,2,1], inplace = True)

In [96]:
X_test = test.iloc[:,0:3]
y_test = test.Sales

In [97]:
pred = DT.predict(X_test)
# RMS = sqrt(mean_squared_error(y_test,pred))
pred

array([2961.        ,  573.        , 1108.        , 3381.        ,
        636.        , 1108.        , 4304.        ,  721.77777778,
       1211.        , 4304.        ,  721.77777778, 1211.        ,
       4304.        ,  721.77777778, 1211.        , 3901.66666667,
        676.70833333, 1211.        , 3901.66666667,  676.70833333,
       1211.        , 3901.66666667,  676.70833333, 1211.        ,
       4457.5       ,  676.70833333, 1048.        , 4457.5       ,
        785.75      , 1145.66666667, 4525.        ,  785.75      ,
       1468.        , 5664.        , 1070.        , 1967.        ])

In [98]:
result_df = pd.DataFrame({'Sales' : pred[:,]})
result_df.to_csv('C:\\Users\\noah\\Desktop\\Unsupervised learning\\Wallmart case study\\result2.csv')